In [1]:
import setup as st
import fx_lib as fx
from datetime import datetime
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 40)
import numpy as np
import itertools
import pickle
import json
import yaml
with open(r'../inp/character_inputs.yaml') as file:
    target_stats = yaml.load(file, Loader=yaml.FullLoader)

with open('../inp/lellian_master.json') as json_file:
    data = json.load(json_file)

import set_combo as sc

In [2]:
df_equipped = sc.df_items[~(sc.df_items.hero == '')].copy()

In [3]:
df_hero_stats = fx.hero_json_to_df(df_equipped.hero.unique(), data)
df_hero_stats['hero'] = df_hero_stats['Name']

In [4]:
for j in range(0,6):
    temp_df = df_hero_stats.merge(df_equipped[['hero','id']][df_equipped.Type == j], \
                        how = 'left', on = 'hero' )
    df_hero_stats[str(j)] = temp_df['id']

In [5]:
xx = sc.df_items.groupby('hero').count().reset_index()
xx[(xx.hero > '')&(xx.efficiency != 6)][['hero','id']]

,hero,id
2,Adlay,5
5,Aither,1
10,Angelica,5
11,Armin,4
12,Assassin Cartuja,5
13,Assassin Cidd,3
19,Blaze Dingo,5
20,Carmainerose,5
21,Celeste,5
22,Celestial Mercedes,2


In [5]:
def get_set_bonus(df, item_df):
    #get data for set combinations
    gears = df[['0','1','2','3','4','5']].values
    set_stats = item_df[item_df.id.isin(gears)].groupby(['set']).count()[['id']]
    set_stats = fx.set_df.merge(set_stats, how='inner', left_on='Set_Nm' , right_on='set')
    set_stats['Mult'] = (set_stats.id / set_stats.Set_Lg).astype(int)
    complete_sets_ind = (set_stats.Mult * set_stats.Set_Lg).sum()
    set_list = []
    for set_nm in set_stats.Set_Nm.values:
        v = set_stats[(set_stats.Set_Nm == set_nm)].iloc[0]['Mult']
        for i in range(0, v):
            set_list.append(set_nm)
    #set info
    df['Complete'] = np.where(complete_sets_ind == 6, 1,0)
    df['Set_1'] = set_list[0] if len(set_list) >= 1 else None
    df['Set_2'] = set_list[1] if len(set_list) >= 2 else None
    df['Set_3'] = set_list[2] if len(set_list) >= 3 else None
    return df

In [6]:
current_gear = df_hero_stats.apply(lambda row: get_set_bonus(row, df_equipped), axis=1)
current_gear[['0','1','2','3','4','5']] = current_gear[['0','1','2','3','4','5']].fillna('z')
current_gear['Gear'] = str("['") + current_gear['0'] + str("','")+current_gear['1'] + str("','")+ \
            current_gear['2']+str("','")+ current_gear['3'] + str("','")+current_gear['4'] + \
            str("','")+current_gear['5'] + str("']")

In [8]:
current_gear[current_gear.Complete == 0]

,Name,Lvl,Artifact,Character,Atk,HP,Speed,Def,Crit Rate,Crit Dmg,Effectiveness,Eff Resist,Element,Role,Specialty Change Runes (Already Applied),Level,SC,EE,HP_flat,Def_flat,Atk_flat,hero,0,1,2,3,4,5,Complete,Set_1,Set_2,Set_3,Gear
7,Adlay,50.0,NaN,Adlay,833,3128,110,489,19,150,0,0,Earth,Mage,NaN,50,0,0,1,0,0,Adlay,man20082404,jtgoypd9,jtunsmzm,jt0tbtno,jtgztlc0,z,0,Counter,None,None,"['man20082404','jtgoypd9','jtunsmzm','jt0tbtno..."
9,Aither,50.0,NaN,Aither,543,3385,92,541,15,150,0,0,Ice,Soul Weaver,NaN,50,0,0,1,0,1,Aither,z,z,aith003,z,z,z,0,None,None,None,"['z','z','aith003','z','z','z']"
14,Angelica,50.0,NaN,Angelica,462,4326,88,598,15,150,0,0,Ice,Soul Weaver,NaN,50,0,0,0,0,1,Angelica,jtrikxqi,jta679sa,jtnhhd0e,z,jtt3un6m,jtxdkoi7,0,Critical,None,None,"['jtrikxqi','jta679sa','jtnhhd0e','z','jtt3un6..."
15,Armin,50.0,"{'Atk': 0.0, 'HP': 0.0, 'Spd': 8.0}",Armin,578,4928,91,633,15,150,0,6,Earth,Knight,NaN,50,0,1,0,0,1,Armin,jt6se80p,z,jtlvf5lc,z,jtamc59k,jtur0otl,0,Defense,None,None,"['jt6se80p','z','jtlvf5lc','z','jtamc59k','jtu..."
16,Assassin Cartuja,50.0,NaN,Assassin Cartuja,854,4850,100,479,15,150,0,0,Dark,Warrior,NaN,50,0,0,0,0,0,Assassin Cartuja,jtfh66da,jtdjsjeb,z,jtcxovpj,jtypfc06,jtshs3se,0,Hit,None,None,"['jtfh66da','jtdjsjeb','z','jtcxovpj','jtypfc0..."
17,Assassin Cidd,50.0,NaN,Assassin Cidd,746,3803,128,401,27,150,0,0,Dark,Thief,NaN,50,0,0,1,1,1,Assassin Cidd,z,jt3bpmjz,jt307x39,z,jtbow5w0,z,0,None,None,None,"['z','jt3bpmjz','jt307x39','z','jtbow5w0','z']"
22,Blaze Dingo,50.0,"{'Atk': 0.0, 'HP': 0.0, 'DefP': 12.0}",Blaze Dingo,659,3321,106,506,15,150,0,0,Light,Soul Weaver,NaN,50,0,1,1,0,1,Blaze Dingo,jt3yqu10,jtkicxvq,jt12mugo,z,jtzu9y43,jthknb5d,0,Speed,None,None,"['jt3yqu10','jtkicxvq','jt12mugo','z','jtzu9y4..."
23,Carmainerose,50.0,NaN,Carmainerose,936,3090,101,537,15,150,6,0,Fire,Mage,NaN,50,0,0,1,0,0,Carmainerose,jt21i4h6,jt56yla0,jtgworwn,z,jtn2lmsq,jtdah8dd,0,Speed,None,None,"['jt21i4h6','jt56yla0','jtgworwn','z','jtn2lms..."
24,Celeste,50.0,NaN,Celeste,744,3771,124,398,15,150,0,0,Light,Ranger,NaN,50,0,0,1,1,1,Celeste,jtjaznxp,z,jt9ym2r2,jtaktvwg,jtl4pzy7,jtk66yz7,0,Critical,None,None,"['jtjaznxp','z','jt9ym2r2','jtaktvwg','jtl4pzy..."
25,Celestial Mercedes,50.0,NaN,Celestial Mercedes,906,3578,101,506,27,160,0,0,Dark,Mage,NaN,50,0,0,1,0,0,Celestial Mercedes,z,z,jtx3mi3d,jtz5jvvd,z,z,0,None,None,None,"['z','z','jtx3mi3d','jtz5jvvd','z','z']"


In [7]:
ee_df = pd.DataFrame(columns= ['Name','Artifact','Atk','AtkP','HP','HPP','Def','DefP','Spd','CChance','CDmg','Eff','Res'])
ee_df['Name'] = current_gear[current_gear.Complete == 1]['Name']
ee_df['Artifact'] = current_gear[current_gear.Complete == 1]['Artifact']
ee_df = ee_df.fillna(0)
for index, row in ee_df.iterrows():
    char=row["Name"]
    for stat in np.unique(fx.gear_rating_lookup.stat_in.values):
        try: 
            val = row["Artifact"][stat]
            ee_df.loc[index,stat] = val
        except: next

In [8]:
def current_stats_fx(current_gear , df_equipped):
    set_df = fx.set_sum(current_gear[current_gear.Complete == 1])
    sub_df = fx.subst_sum(current_gear[current_gear.Complete == 1], df_equipped)
    main_df = fx.mainst_sum(current_gear[current_gear.Complete == 1], df_equipped)
    df = pd.DataFrame()
    df_hero = current_gear[current_gear.Complete == 1].copy().sort_values(by = ['Name']).reset_index()
    mainst_df = main_df.copy().sort_values(by = ['Name']).reset_index()
    subst_df = sub_df.copy().sort_values(by = ['Name']).reset_index()
    setst_df = set_df.copy().sort_values(by = ['Name']).reset_index()
    hero_ee = ee_df.copy().sort_values(by = ['Name']).reset_index()
    df['Char'] = df_hero['Name']
    df['Level'] = df_hero['Level']
    df['Set_1'] = df_hero['Set_1']
    df['Set_2'] = df_hero['Set_2']
    df['Set_3'] = df_hero['Set_3']
    df['ATK'] =  (df_hero['Atk']   *(100+mainst_df['ATK%']+subst_df['ATK%']+setst_df['ATK']+hero_ee['AtkP'])/100  +mainst_df['ATK']+subst_df['ATK']+hero_ee['Atk']).astype(int)
    df['HP'] =   (df_hero['HP']    *(100+mainst_df['HP%']+subst_df['HP%']+setst_df['HP']+hero_ee['HPP'])/100   +mainst_df['HP'] +subst_df['HP']+hero_ee['HP']).astype(int)
    df['DEF'] =  (df_hero['Def']   *(100+mainst_df['DEF%']+subst_df['DEF%']+setst_df['DEF']+hero_ee['DefP'])/100 +mainst_df['DEF'] +subst_df['DEF']+hero_ee['Def']).astype(int)
    df['SPD'] =  (df_hero['Speed'] * ((100+setst_df['SPD'].values)/100) +mainst_df['SPD']+subst_df['SPD']+hero_ee['Spd'] ).astype(int)
    df['CRIT'] = (df_hero['Crit Rate'] +mainst_df['CRIT']+subst_df['CRIT']+setst_df['CRIT']+hero_ee['CChance']).astype(int)
    df['CDMG'] = (df_hero['Crit Dmg'].values[0]  +mainst_df['CDMG']+subst_df['CDMG']+setst_df['CDMG']+hero_ee['CDmg']).astype(int)
    df['EFF'] =  (df_hero['Effectiveness'] +mainst_df['EFF']+subst_df['EFF']+setst_df['EFF']+hero_ee['Eff']).astype(int)
    df['RES'] =  (df_hero['Eff Resist']    +mainst_df['RES']+subst_df['RES']+setst_df['RES']+hero_ee['Res']).astype(int)
    cp1 = round( ((df['ATK']*1.6 + df['ATK']*1.6*df['CRIT']*df['CDMG']/100/100) * (1+(df['SPD']-45)*0.02) + df['HP'] + df['DEF']*9.3) * (1 + (df['RES']+df['EFF']) / 400) , 0)
    cp2 = 1 + 0.08 * df_hero['SC'].values[0] + 0.02 * df_hero['EE'].values[0] + 0.03
    df['CP'] = round(cp1 * cp2,0).astype(int)
    df['Dmg_Rating'] = ((df['ATK']/ 2500 \
            * (df['CRIT']/100 * df['CDMG']/100 + (100-df['CRIT'])/100) \
            * df['SPD'] / 150)*10).astype(int)
    df['EHP'] = (df['HP'] * (1 + df['DEF']/300) / 100).astype(int)
    df['CMult'] = df['CRIT'] / 100 * df['CDMG'] / 100
    df['CATK'] = df['ATK'] / 100 * df['CDMG'] / 100
    df['WR'] = ((df['ATK']/1500 + df['SPD']/100 + df['CRIT']/30 + df['CDMG']/150 \
                + df['HP']/5000 + df['DEF']/400 + df['EFF']/30 + df['RES']/30)*10).astype(int)
    df['PI'] = (df['ATK']/df_hero['Atk'].values[0]/fx.grl['max_t7'][0] \
                + df['SPD']/df_hero['Speed'].values[0]/fx.grl['max_t7'][2] \
                + (df['CRIT'] - df_hero['Crit Rate'].values[0])/100/fx.grl['max_t7'][3] \
                + (df['CDMG']-df_hero['Crit Dmg'].values[0])/100/fx.grl['max_t7'][4] \
                + df['HP']/df_hero['HP'].values[0]/fx.grl['max_t7'][5] \
                + df['DEF']/df_hero['Def'].values[0]/fx.grl['max_t7'][7] \
                + (df['EFF'] - df_hero['Effectiveness'].values[0])/100/fx.grl['max_t7'][9] \
                + (df['RES'] - df_hero['Eff Resist'].values[0])/100/fx.grl['max_t7'][10]).astype(int)
    df['Element'] = df_hero['Element']
    df['Role'] = df_hero['Role']
    return df

In [9]:
st.GEAR_12 = 0
df1 = current_stats_fx(current_gear , df_equipped)
st.GEAR_12 = 1
df2 = current_stats_fx(current_gear , df_equipped)

In [12]:
# current gear
# df1.to_csv('../reco/hero_stats_old.csv')
df1.sort_values(by = ['EHP'], ascending = False)

,Char,Level,Set_1,Set_2,Set_3,ATK,HP,DEF,SPD,CRIT,CDMG,EFF,RES,CP,Dmg_Rating,EHP,CMult,CATK,WR,PI,Element,Role
43,Krau,60,Hit,Defense,Health,1418,23203,1592,151,51,166,20,52,63288,7,1463,0.8466,23.5388,162,119,Ice,Knight
66,Tywin,60,Speed,Health,None,1264,21367,1187,190,22,174,77,29,57531,7,1059,0.3828,21.9936,154,105,Ice,Knight
6,Angelic Montmorancy,60,Speed,Health,None,1179,19950,1284,195,27,164,11,133,61087,7,1053,0.4428,19.3356,167,106,Ice,Soul Wevaer
17,Charles,60,Counter,Immunity,None,1803,15938,1251,166,56,197,31,23,57588,12,823,1.1032,35.5191,141,107,Earth,Knight
54,Ravi,60,Hit,Counter,None,2988,14345,1371,122,28,164,87,27,60466,11,799,0.4592,49.0032,153,125,Fire,Warrior
32,General Purrgis,50,Speed,Defense,None,1443,14690,1306,151,78,150,57,8,51836,8,786,1.1700,21.6450,144,97,Light,Warrior
38,Ken,50,Health,Counter,None,1377,16997,1032,143,55,162,19,7,43530,7,754,0.8910,22.3074,121,90,Fire,Warrior
29,Falconer Kluri,60,Speed,Hit,None,1408,11301,1681,201,15,162,61,5,47060,8,746,0.2430,22.8096,131,86,Earth,Knight
11,Benevolent Romann,60,Speed,Critical,None,1622,13402,1363,186,56,160,73,29,59133,10,742,0.8960,25.9520,153,100,Light,Mage
26,Dizzy,60,Speed,Health,None,2254,14972,1183,226,39,150,96,50,75077,16,740,0.5850,33.8100,168,113,Ice,Mage


In [ ]:
# gear at least+12
# df2.to_csv('../reco/hero_stats_old.csv')
df2.sort_values(by = ['CP'], ascending = False)

EQUIPPED GEAR STATS

In [ ]:
df_equipped[(df_equipped.enhance < 15)&(df_equipped.level == 85)]\
    .sort_values(by = ['maxp'], ascending = False)

In [ ]:
# df_equipped[(df_equipped.hero.isin(['Vildred','Challenger Dominiel','Tamarinne','Kayron']))]
df_equipped[(df_equipped.hero.isin(['Faithless Lidica','Dizzy','Basar','Landy','Kayron','Tenebria']))]

In [ ]:
df_equipped2 = df_equipped.copy()
df_equipped['hero'] = np.where(df_equipped.id == '' , '' , df_equipped.hero)

GEAR STATS

In [ ]:
sc.df_items[~(sc.df_items.set.isin(['Lifesteal']))&~(sc.df_items.slot == 'Boots')].sort_values(by = ['SPD'], ascending = False)

In [ ]:
sc.df_items[(sc.df_items.Type == 0)&(sc.df_items.hero=='')] \
    .groupby(['level','enhance'])[['current_pct','current_eff']].agg([np.max, np.mean, np.min])

Hero's gear

In [11]:
df_equipped[df_equipped.hero=='Angelic Montmorancy']

,efficiency,enhance,hero,id,level,locked,mainStat,p_id,rarity,set,slot,subStat1,subStat2,subStat3,subStat4,grade,Type,ATK%,ATK,SPD,...,CDMG,HP%,HP,DEF%,DEF,EFF,RES,main_tp,main_val,GR,GR2,minp,maxp,spdp,rating,efficiency2,max_eff,current_eff,current_pct,reco
331,85,15,Angelic Montmorancy,jtsgy9gx,85,False,"[Atk, 500]",id201810,Epic,Speed,Weapon,"[Spd, 6]","[Res, 18]","[HPP, 19]","[Eff, 7]",0,0,0,0,6,...,0,19,0,0,0,7,18,Atk,500,0.676538,0.676538,0.676538,0.676538,6,2.024538,20,85,8.513149,8.513149,
332,86,15,Angelic Montmorancy,jtdxihlx,88,False,"[HP, 2765]",id105524,Epic,Speed,Helmet,"[HPP, 16]","[DefP, 10]","[Spd, 4]","[Res, 19]",0,1,0,0,4,...,0,16,0,10,0,0,19,HP,2765,0.644439,0.644439,0.644439,0.644439,4,2.084439,20,86,8.653297,8.653297,
333,87,15,Angelic Montmorancy,jtard345,85,False,"[Def, 300]",id000351,Epic,Speed,Armor,"[HPP, 24]","[Spd, 7]","[CDmg, 14]","[Eff, 4]",0,2,0,0,7,...,14,24,0,0,0,4,0,Def,300,0.695676,0.695676,0.695676,0.695676,7,2.053676,20,87,8.639459,8.639459,
334,87,15,Angelic Montmorancy,jt95lnte,75,False,"[HPP, 60]",id231420,Epic,Health,Necklace,"[DefP, 7]","[Spd, 7]","[CChance, 4]","[Res, 31]",0,3,0,0,7,...,0,0,0,7,0,0,31,HPP,60,0.698761,0.698761,0.698761,0.698761,7,2.056761,20,87,8.659822,8.659822,
335,83,15,Angelic Montmorancy,jt6owm8q,75,False,"[HPP, 60]",id231414,Epic,Health,Ring,"[DefP, 18]","[Spd, 9]","[CChance, 4]","[Res, 11]",0,4,0,0,9,...,0,0,0,18,0,0,11,HPP,60,0.632095,0.632095,0.632095,0.632095,9,2.010095,20,83,8.219829,8.219829,
336,90,15,Angelic Montmorancy,jtsocy2e,85,False,"[Spd, 45]",id194610,Heroic,Speed,Boots,"[CChance, 4]","[HPP, 30]","[Res, 14]","[AtkP, 9]",1,5,9,0,0,...,0,30,0,0,0,0,14,Spd,45,0.728389,0.728389,0.728389,0.728389,0,2.216389,22,90,8.855365,8.855365,


### Gear analysis

In [ ]:
df_lvl85 = sc.df_items[(sc.df_items.level == 85)].copy()

In [ ]:
## look at level 85 gear
df_lvl85.groupby(['level','enhance'])[['efficiency','maxp']].agg(['count','max','mean','min']) 

In [ ]:
# df_lvl85[(df_lvl85.slot == 'Boots')&(df_lvl85.main_tp == 'Spd')].sort_values(by=['efficiency'], ascending = False)
df_lvl85[(df_lvl85.enhance < 15)&(df_lvl85.SPD > 0)].sort_values(by=['maxp'], ascending = False)

#### Gear var

In [ ]:
# gear_chk = sc.df_items[(sc.df_items.main_tp == 'Spd')&(sc.df_items.enhance<15)].copy()
gear_chk = sc.df_items[(sc.df_items.level >= 75)&(sc.df_items.Type.isin([3,4]))].copy()

In [ ]:
gear_chk.groupby(['slot','main_tp','set'])[['efficiency','maxp']].agg(['count','max','mean','min']) 

#### Gear list

In [ ]:
# gear_chk = sc.df_items[(sc.df_items.main_tp == 'Spd')&(sc.df_items.enhance<15)].copy()
gear_chk = sc.df_items[(sc.df_items.level >= 75)&(sc.df_items.enhance<15)].copy()

In [ ]:
gear_chk.sort_values(['SPD'],ascending=False)

### Gear removal

##### choose gear to delete

In [ ]:
df_items = pd.read_pickle('../outp/equip_potential.pkl')

In [ ]:
x1 = df_items['efficiency'].quantile(q=0.4)
x2 = df_items['max_eff'].quantile(q=0.3)
x3 = df_items['rating'].quantile(q=0.4)
x4 = df_items['current_eff'].quantile(q=0.3)

In [ ]:
low_items = df_items[(df_items.efficiency <= x1) & (df_items.max_eff <= x2) & (df_items.rating <= x3) & (df_items.current_eff <= x4) & (df_items.SPD <= 8)].copy()


In [ ]:
print("number of items" , len(df_items))
print("number of unequipped items" , len(df_items[df_items.hero=='']))
print("number of items to delete" , len(low_items))
print("number of unequipped items to delete" , len(low_items[low_items.hero=='']))

In [ ]:
low_items

In [ ]:
low_items[low_items.hero==''][['id','p_id','efficiency','max_eff','set','slot','level','enhance']].to_csv('../outp/remove_list.csv')

##### remove gear from items

In [ ]:
remove = pd.read_csv('../outp/remove_list.csv')

In [ ]:
check1 = len(remove.id.values)
check2 = df_items[df_items.id.isin(remove.id.values)]['id'].count()

In [ ]:
if check1 == check2:
    print('pass', check1, check2)
    df_items = df_items.sort_values(by = ['hero','Type','efficiency','enhance'])
    export1 = df_items[~df_items.id.isin(remove.id.values)][['efficiency','hero','enhance','slot','level','set','rarity','mainStat','subStat1','subStat2','subStat3','subStat4','id','p_id','locked']].copy()
    export2 = export1.to_dict('records')
    with open('../outp/upd_items.json', 'w') as fp: json.dump(export2, fp)

In [ ]:
print( sc.df_items['id'].count() , export1['id'].count(), check1 )